In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBRegressor
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings('ignore')


In [ ]:
df_train = pd.read_csv("../input/san-francisco-crime-classification/train.csv")
df_test = pd.read_csv("../input/san-francisco-crime-classification/test.csv", index_col="Id")

def impute(df):
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    d = (df.dtypes != 'object')
    object_num = list(d[d].index)
    for name in object_cols:
        df[name]=df[name].fillna("none")
    for name in object_num:
        df[name]=df[name].fillna(0)
    return df






In [ ]:
df_train.Category.unique()

In [ ]:
df_train[['date','time']]=df_train.Dates.str.split(expand=True)
df_train['Year'] = pd.DatetimeIndex(df_train['date']).year
df_train= df_train.drop(['date','Dates','Descript','Resolution'],axis=1)
df_train.head()
df_test[['date','time']]=df_test.Dates.str.split(expand=True)
df_test['Year'] = pd.DatetimeIndex(df_test['date']).year
df_test= df_test.drop(['date','Dates'],axis=1)
df_test.head()


In [ ]:


from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

  

df_train['Category']= label_encoder.fit_transform(df_train['Category'])
df_train['DayOfWeek']= label_encoder.fit_transform(df_train['DayOfWeek'])
df_train['PdDistrict']= label_encoder.fit_transform(df_train['PdDistrict'])
df_train['Address']= label_encoder.fit_transform(df_train['Address'])
df_train['time']= label_encoder.fit_transform(df_train['time'])

df_test['DayOfWeek']= label_encoder.fit_transform(df_test['DayOfWeek'])
df_test['PdDistrict']= label_encoder.fit_transform(df_test['PdDistrict'])
df_test['Address']= label_encoder.fit_transform(df_test['Address'])
df_test['time']= label_encoder.fit_transform(df_test['time'])




In [ ]:
X = df_train
X = X.rename(columns={"X":"Longitude"})
X = X.rename(columns={"Y":"Latitude"})
y = X.pop("Category")

features = [
    "Longitude",
    "Latitude"
]

# Standardize
X_scaled = X.loc[:,features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)

kmeans = KMeans(n_clusters=10, n_init=10, random_state=0)
X["Cluster"] = kmeans.fit_predict(X_scaled)
X1 = df_test
X1 = X1.rename(columns={"X":"Longitude"})
X1 = X1.rename(columns={"Y":"Latitude"})


features = [
    "Longitude",
    "Latitude"
]

# Standardize
X_scaled_1 = X1.loc[:,features]
X_scaled_1 = (X_scaled_1 - X_scaled_1.mean(axis=0)) / X_scaled_1.std(axis=0)

kmeans = KMeans(n_clusters=10, n_init=10, random_state=0)
X1["Cluster"] = kmeans.fit_predict(X_scaled_1)

In [ ]:
Xy = X
Xy["Cluster"] = Xy.Cluster.astype("category")
Xy["Category"] = y
sns.relplot(
    x="value", y="Category", hue="Cluster", col="variable",
    height=12, aspect=1, facet_kws={'sharex': False}, 
    data=Xy.melt(
        value_vars=features, id_vars=["Category", "Cluster"],
    ),
);

In [ ]:
X.head()

In [ ]:
import numpy as np
from collections import Counter
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,accuracy_score
pd.options.display.max_columns = 999



In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X.copy(),y.copy(),test_size=0.3,random_state=0)

In [ ]:
d_train=lgb.Dataset(X_train, label=y_train)
params={}
params['learning_rate']=0.01
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=40 #no.of unique values in the target class not inclusive of the end value
params['num_leaves'] = 700
clf=lgb.train(params,d_train,100)

In [ ]:
y_pred_1=clf.predict(X_test)
y_pred_1 = [np.argmax(line) for line in y_pred_1]


In [ ]:
accuracy_score(y_test,y_pred_1)

In [ ]:
d_train=lgb.Dataset(X, label=y)
params={}
params['learning_rate']=0.01
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=40 #no.of unique values in the target class not inclusive of the end value
params['num_leaves'] = 700
clf=lgb.train(params,d_train,100)

In [ ]:
X1.head()


In [ ]:
y_pred_2=clf.predict(X1)
y_pred_2 = [np.argmax(line) for line in y_pred_2]


In [ ]:
output = pd.DataFrame({'Id': X1.index, 'Category': y_pred_2})
output.to_csv('my_submission2.csv', index=False)

In [ ]:
df_predictions=pd.read_csv("../input/predicts/my_submission2_FINAL2.csv")
one_hot_encoded_data = pd.get_dummies(df_predictions, columns = ['Category'])
one_hot_encoded_data.to_csv('my_submission2_FINAL.csv', index=False)
